In [129]:
import pandas
from datetime import time
pandas.set_option('display.max_columns', 500)

In [130]:
# df = pandas.read_excel('minute_candle.xlsx',dtype=str)

In [131]:
import sqlite3
with sqlite3.connect("FastDb") as db:
    df = pandas.read_sql_query('select * from minute_candle',db)

In [132]:
df.InstrumentIdentifier.unique()

array(['ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK',
       'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL',
       'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY',
       'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE',
       'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'ITC',
       'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LTIM', 'LT', 'M&M',
       'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE',
       'SBILIFE', 'SBIN', 'SUNPHARMA', 'TCS', 'TATACONSUM', 'TATAMOTORS',
       'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'WIPRO'],
      dtype=object)

In [133]:
df.columns

Index(['CreatedOn', 'Exchange', 'InstrumentIdentifier', 'LastTradeTime',
       'ServerTime', 'AverageTradedPrice', 'BuyPrice', 'BuyQty', 'CloseValue',
       'High', 'Low', 'LastTradePrice', 'LastTradeQty', 'OpenValue',
       'OpenInterest', 'QuotationLot', 'SellPrice', 'SellQty',
       'TotalQtyTraded', 'ValueDataPoint', 'PreOpen', 'PriceChange',
       'PriceChangePercentage', 'OpenInterestChange', 'MessageType'],
      dtype='object')

In [134]:
def seg_data(df,symbol='SBIN',start_h=9,start_m=15,end_h=15,end_m=30):
    start_time = time(start_h,start_m)
    end_time = time(end_h,end_m)
    dftouse = df[df['InstrumentIdentifier']==symbol].copy()
    dftouse['CreatedOn']=pandas.to_datetime(dftouse['CreatedOn']).copy()
    filtered_df = dftouse[(dftouse['CreatedOn'].dt.time >= start_time) & (dftouse['CreatedOn'].dt.time <= end_time)]
    filtered_df.reset_index(inplace=True, drop=True)
    return filtered_df

In [145]:
dd = seg_data(df)
dd = dd[['CreatedOn','High','Low','OpenValue','CloseValue']].copy()
dd.set_index('CreatedOn',inplace =True)
dd

,High,Low,OpenValue,CloseValue
CreatedOn,,,,
2023-07-28 09:15:00,617.15,615.70,617.15,615.75
2023-07-28 09:16:01,616.40,615.55,616.40,615.85
2023-07-28 09:17:00,616.15,615.10,616.10,615.50
2023-07-28 09:18:00,615.80,615.05,615.50,615.45
2023-07-28 09:19:00,615.75,614.70,615.50,615.45
...,...,...,...,...
2023-08-11 13:28:00,577.20,577.00,577.05,577.20
2023-08-11 13:29:00,577.25,577.05,577.10,577.10
2023-08-11 13:30:02,577.00,576.50,577.00,576.50


In [136]:
dd.isna().sum()

CreatedOn     0
High          0
Low           0
OpenValue     0
CloseValue    0
dtype: int64

In [137]:
def bb(df,window=20,std=1):
    curr_df = pandas.DataFrame()
    curr_df['MB'] = df['CloseValue'].rolling(window=window).mean()
    curr_df['std'] =( df['CloseValue'].rolling(window=window).std())*std
    curr_df['UB'] = curr_df['MB'] + curr_df['std']
    curr_df['LB'] = curr_df['MB'] - curr_df['std']
    return curr_df['UB'],curr_df['MB'],curr_df['LB']

In [138]:
dd['UB'],dd['MB'],dd['LB'] = bb(dd)

In [139]:
dd

,CreatedOn,High,Low,OpenValue,CloseValue,UB,MB,LB
0,2023-07-28 09:15:00,617.15,615.70,617.15,615.75,NaN,NaN,NaN
1,2023-07-28 09:16:01,616.40,615.55,616.40,615.85,NaN,NaN,NaN
2,2023-07-28 09:17:00,616.15,615.10,616.10,615.50,NaN,NaN,NaN
3,2023-07-28 09:18:00,615.80,615.05,615.50,615.45,NaN,NaN,NaN
4,2023-07-28 09:19:00,615.75,614.70,615.50,615.45,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3739,2023-08-11 13:28:00,577.20,577.00,577.05,577.20,577.761475,577.3325,576.903525
3740,2023-08-11 13:29:00,577.25,577.05,577.10,577.10,577.689100,577.2875,576.885900
3741,2023-08-11 13:30:02,577.00,576.50,577.00,576.50,577.579983,577.2025,576.825017
3742,2023-08-11 13:31:00,576.50,576.35,576.50,576.40,577.482520,577.1200,576.757480


In [115]:
def change_df_tf(x_minutes = 5):
    resampled_df = dd.resample(f'{x_minutes}T').agg({
        'OpenValue': 'first',
        'High': 'max',
        'Low': 'min',
        'CloseValue': 'last'
    })

    return resampled_df

In [121]:
dd.to_excel('Sorted_Col_One.xlsx')

In [119]:
dd_dict = dd.to_dict('index')

In [120]:
dd_dict

{Timestamp('2023-07-28 09:15:00'): {'High': 7318.0,
  'Low': 7292.45,
  'OpenValue': 7318.0,
  'CloseValue': 7300.15,
  'UB': nan,
  'MB': nan,
  'LB': nan},
 Timestamp('2023-07-28 09:16:01'): {'High': 7328.8,
  'Low': 7304.8,
  'OpenValue': 7306.2,
  'CloseValue': 7320.0,
  'UB': nan,
  'MB': nan,
  'LB': nan},
 Timestamp('2023-07-28 09:17:00'): {'High': 7326.8,
  'Low': 7313.9,
  'OpenValue': 7320.0,
  'CloseValue': 7324.6,
  'UB': nan,
  'MB': nan,
  'LB': nan},
 Timestamp('2023-07-28 09:18:00'): {'High': 7334.4,
  'Low': 7314.85,
  'OpenValue': 7326.5,
  'CloseValue': 7334.4,
  'UB': nan,
  'MB': nan,
  'LB': nan},
 Timestamp('2023-07-28 09:19:00'): {'High': 7333.5,
  'Low': 7323.1,
  'OpenValue': 7333.45,
  'CloseValue': 7333.0,
  'UB': nan,
  'MB': nan,
  'LB': nan},
 Timestamp('2023-07-28 09:20:00'): {'High': 7335.0,
  'Low': 7305.0,
  'OpenValue': 7335.0,
  'CloseValue': 7305.0,
  'UB': nan,
  'MB': nan,
  'LB': nan},
 Timestamp('2023-07-28 09:21:00'): {'High': 7309.85,
  'Low'